In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [3]:
file = '/home/mango/Documents/unipd/Earthquakes/SouthCalifornia-1982-2011_Physics-of-Data.dat' 
data = np.genfromtxt(file, skip_header=0, skip_footer=1, 
                     names=True, dtype=None, delimiter=' ')         
header = ["id","id_prv","time","magnitude","x","y", "z"]
df = pd.DataFrame(data)
df.columns = header
df.shape

(110269, 7)

In [4]:
#plt.plot(df["time"], df["magnitude"])
#plt.plot(df["time"], df["x"])
#plt.plot(df["time"], df["y"])
#plt.plot(df["time"], df["z"])
#plt.plot(df["time"], df["id_prv"])


In [5]:
df.describe()

,id,id_prv,time,magnitude,x,y,z
count,110269.000000,110269.000000,1.102690e+05,110269.000000,1.102690e+05,1.102690e+05,1.102690e+05
mean,55135.000000,35859.928040,4.262419e+08,2.457910,-2.388222e+06,-4.691075e+06,3.562611e+06
std,31832.062754,36365.010382,2.598553e+08,0.438371,8.877740e+04,1.237274e+05,1.271892e+05
min,1.000000,-1.000000,3.650139e+04,2.000000,-2.770239e+06,-5.020849e+06,3.186133e+06
25%,27568.000000,-1.000000,2.115798e+08,2.140000,-2.421613e+06,-4.801153e+06,3.458836e+06
50%,55135.000000,28462.000000,3.551023e+08,2.340000,-2.361828e+06,-4.699729e+06,3.564367e+06
75%,82702.000000,66974.000000,5.952375e+08,2.640000,-2.329898e+06,-4.615063e+06,3.634542e+06
max,110269.000000,110250.000000,9.305363e+08,7.300000,-1.998294e+06,-4.268289e+06,3.876120e+06


In [16]:
def bins_histogram(data, precision=0.01):
    """
    Params
    
    data: 1d numpy array
    precision: desired poisson precision of bins
    """
    N = int(pow(1/precision,2))
    data.sort()
    x = data[::N]
    y = data[1::N]
    if y.shape[0] == x.shape[0]:
        bins = np.concatenate((np.array(data.min()[np.newaxis]),(x[1:] + y[1:])/2, np.array(data.max()[np.newaxis])))
    else:
        bins = np.concatenate((np.array(data.min()[np.newaxis]),(x[1:-1] + y[1:])/2, np.array(data.max()[np.newaxis])))
    return bins

In [17]:
data = np.random.rand(100000)
bins = p_histogram(data)
print(bins)
print(data)


[1.22694862e-05 1.00384288e-01 1.99919742e-01 2.99484822e-01
 4.01228069e-01 5.02255966e-01 6.03793427e-01 7.01866388e-01
 8.01654206e-01 9.01442879e-01 9.99999638e-01]
[1.22694862e-05 1.23638915e-05 2.62215176e-05 ... 9.99979773e-01
 9.99990548e-01 9.99999638e-01]
